In [2]:
import torch 
import torch.nn as nn
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import pandas as pd


In [3]:

data = pd.read_table('fra.txt', header= None)

In [4]:
data.rename(columns= {0: 'English', 1: 'French', 2: 'Citation'}, inplace= True)

In [5]:
pattern = r"[!'#$%&()*+,-./:;<=>?@[\]^`{|}~“”‘’«»‹›„‚–—…·•¡¿’\"\']"

eng_sent, french_sent = [], []

for e in range(len(data['English'])):
    eng_sent.append(re.sub(pattern, "", data['English'][e]))
    french_sent.append(re.sub(pattern, "", data['French'][e]))

In [6]:
print(len(eng_sent))
print(len(french_sent))

229803
229803


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
eng_token = [tokenizer.encode(text, add_special_tokens= True) for text in eng_sent]
fren_token = [tokenizer.encode(text, add_special_tokens= True) for text in french_sent]

In [9]:
model = BertModel.from_pretrained('bert-base-uncased')


In [18]:
def padding(tokens, max_len):
    temp = []
    for token in tokens:
        if max_len > len(token):
            temp.append([token + [tokenizer.pad_token_id 
                                 for i in range(max_len - len(tokens))]])
        else:
            temp.append([token[:max_len]])

    return temp

In [11]:
def text_embedding(batch_tokens, max_len):
    batch_padded_tokens = padding(batch_tokens, max_len)
    tokens_tensor = torch.tensor(batch_padded_tokens)
    with torch.no_grad():
        output = model(tokens_tensor)
        embeddings = output.last_hidden_state
    return embeddings

In [12]:
eng_sum = 0
for tokens in eng_token:
    eng_sum += len(tokens)

eng_avg_len = eng_sum // len(eng_token)
print("Average len: {}".format(eng_avg_len))

Average len: 8


In [13]:
fren_sum = 0
for tokens in fren_token:
    fren_sum += len(tokens)

fren_avg_len = fren_sum // len(fren_token)

print("Average len: {}".format(fren_avg_len))

Average len: 12


In [20]:
batch_size = 1000

eng_embedding = []
for i in tqdm(range(0, len(eng_token), batch_size), "Embedding", colour= "green"):
    batch_token = eng_token[i : i+batch_size]
    eng_embedding.extend(text_embedding(batch_token, eng_avg_len))

len(eng_embedding)

Embedding:   0%|          | 0/230 [00:00<?, ?it/s]


ValueError: expected sequence of length 3 at dim 1 (got 4)

In [17]:
batch_size = 10000

fren_embedding = []
for i in tqdm(range(0, len(fren_token), batch_size), "Embedding", colour= "green"):
    batch_token = fren_token[i : i+batch_size]
    fren_embedding.extend(text_embedding(batch_token, fren_avg_len))

len(fren_embedding)

Embedding:   0%|          | 0/23 [00:00<?, ?it/s]

ValueError: expected sequence of length 3 at dim 2 (got 4)